## Imports

In [1]:
import pandas as pd
import numpy as np
import nltk
nltk.download('wordnet')
import re
from bs4 import BeautifulSoup

c:\Users\reuel\anaconda3\envs\pytorch\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\reuel\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

## Read Data

In [3]:
dataframe = pd.read_csv("dataset/amazon_reviews_us_Beauty_v1_00.tsv", sep="\t", on_bad_lines='skip')

C:\Users\reuel\AppData\Local\Temp\ipykernel_19364\47645999.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  dataframe = pd.read_csv("dataset/amazon_reviews_us_Beauty_v1_00.tsv", sep="\t", on_bad_lines='skip')


## Keep Reviews and Ratings

In [4]:
df = dataframe.loc[:, ['star_rating', 'review_body', 'review_headline']]
df['review_body'] = df['review_body'].astype(str)
df['review_headline'] = df['review_headline'].astype(str)

 ## We form three classes and select 20000 reviews randomly from each class.



In [5]:
def assignClass(rating):
    if rating == 1 or rating == 2:
        return 1
    elif rating == 3:
        return 2
    elif rating == 4 or rating == 5:
        return 3
    return -1

In [6]:
df['class'] = df['star_rating'].map(assignClass)

In [7]:
df_1 = df[df["class"] == 1].sample(n=20000)
df_2 = df[df["class"] == 2].sample(n=20000)
df_3 = df[df["class"] == 3].sample(n=20000)
# print(len(df_1), len(df_2), len(df_3))

In [8]:
df = pd.concat([df_1, df_2, df_3])

In [9]:
print("Dataframe that is being worked on:")
df.head(10)

Dataframe that is being worked on:


,star_rating,review_body,review_headline,class
4550602,1,I am so upset and wanted to return. However th...,picture shows 17 brushes.the titel says 13 pc ...,1
2368402,2,horrid name! kids like but why the name?,kids like but why the name,1
2184565,2,Its very thick and I could have gotten it chea...,Two Stars,1
3878729,1,Do not order this product it is not the origin...,FAKE!!!,1
3013017,1,Ive waxed before so im not new to painfully ri...,DONT BUY THIS!!,1
3635167,1,The scarf does not look anything like the pict...,DOES NOT LOOK LIKE THE PICTURE!,1
3572617,1,"A friend bought some of this sunscreen for me,...",Very bad experience,1
3530113,2,I developed a fondness for the June Jacobs Gre...,Underwhelmed,1
4966424,2,"This stuff works, but be careful! it actually ...",burned my gum,1
4183200,2,Doesn't heat up enough. I had it on my hair f...,Leaves much to be desired,1


In [10]:
# calculate length of raw reviews before any cleaning or preprocessing
len_before_cleaning = (df['review_headline'] + ". " + df['review_body']).apply(len).mean()

# Data Cleaning



In [11]:
import contractions

In [12]:
def clean_data(text):
    text = str(text)

    # convert text to lowercase
    text = text.lower()

    # remove urls                                                                                                             
    text = re.sub(r'(http(s)?:\/\/.)?(www\.)?[-a-zA-Z0-9@:%._\+~#=]{2,256}\.[a-z]{2,6}\b([-a-zA-Z0-9@:%_\+.~#?&//=]*)',' ',text)
    # remove email ids    
    text = re.sub(r'([a-zA-Z0-9._-]+@[a-zA-Z0-9._-]+\.[a-zA-Z0-9_-]+)',' ',text)       
    # html tag                                             
    text = re.sub('<[^<]+?>', '', text)                                                                                            
    
    # expand contractions
    text = contractions.fix(text)
    
    # replace non aplha numeric characters
    text = re.sub(r'[^a-zA-Z0-9. ]',' ',text)                                                                                      
    # remove isolated characters
    text = re.sub(r'\s+[a-zA-Z]\s+', ' ', text)
    # remove consecutively repeating words                                                                                     
    text = re.sub(r'\b(\w+)(?:\W+\1\b)+', r'\1', text, flags=re.IGNORECASE)
    # replace every word following not/never/no as NEG_word until a fullstop is found
    text = re.sub(r'\b(?:not|never|no)\b[\w\s]+[^\w\s]',lambda match: re.sub(r'(\s+)(\w+)', r'\1NEG_\2', match.group(0)), text, flags=re.IGNORECASE)

    # remove extra spaces
    text = re.sub(r'\s+', ' ', text)                                                                                                

    return text

In [13]:
review_body = df["review_body"].apply(clean_data)
review_headline = df["review_headline"].apply(clean_data)

#### Comparing length of raw uncleaned review with length of cleaned review 

In [14]:
len_after_cleaning = (review_headline + ". " + review_body).apply(len).mean()
print(f"{len_before_cleaning}, {len_after_cleaning}")


313.3815, 337.23428333333334


In [15]:
# store the cleaned data that will NOT be pre-processed
reviews_vanilla = review_headline.str.upper() + ". " + review_body

# Pre-processing

## remove the stop words 

In [16]:
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = stopwords.words('english')
stop_words.remove('not')

def remove_stop_words(text):
    tokens = [w for w in text.split() if not w in stop_words]

    
    text = " ".join(tokens)

    return text

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\reuel\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [17]:
review_body = review_body.apply(remove_stop_words)
review_headline = review_headline.apply(remove_stop_words)

## perform lemmatization  

In [18]:
from nltk.corpus import wordnet

def get_wordnet_pos(treebank_tag):

    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return ''

In [19]:
from nltk.stem import WordNetLemmatizer
from nltk.tag import pos_tag
nltk.download('averaged_perceptron_tagger')
nltk.download('omw-1.4')

lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\reuel\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\reuel\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [20]:
def lemmatization(reviews):
    lemmatized_reviews = []
    for sent in reviews.values:
        tagged = pos_tag(sent.split(" "))
        temp_review = ""
        for token in tagged:
            tag = get_wordnet_pos(token[1])
            if tag:
                temp_review += lemmatizer.lemmatize(token[0], tag)
            else:
                temp_review += token[0]
            temp_review += " "

        lemmatized_reviews.append(temp_review)

    return  pd.Series(lemmatized_reviews)

In [21]:
review_body = lemmatization(review_body)
review_headline = lemmatization(review_headline)

In [22]:
# store the cleaned and pre-processed data
reviews_processed = review_headline.str.upper() + ". " + review_body

#### Comparing length of review that is NOT preprocessed with length of review before preprocessing

In [23]:
len_without_preprocessing = reviews_vanilla.apply(len).mean()

print(f"{len_after_cleaning}, {len_without_preprocessing}")

337.23428333333334, 337.23428333333334


In [24]:
print("Sample of cleaned Data:")
reviews_vanilla.head(10)

Sample of cleaned Data:


4550602    PICTURE SHOWS 17 TITEL SAYS 13 PC AND RECEIVED...
2368402    KIDS LIKE BUT WHY THE NAME. horrid name kids l...
2184565    TWO STARS. its very thick and could have gotte...
3878729    FAKE . do not NEG_order NEG_this NEG_product N...
3013017    DO NOT BUY THIS . i have waxed before so am no...
3635167    DOES NOT LOOK LIKE THE PICTURE . the scarf doe...
3572617    VERY BAD EXPERIENCE. a friend bought some of t...
3530113    UNDERWHELMED. i developed fondness for the jun...
4966424    BURNED MY GUM. this stuff works but be careful...
4183200    LEAVES MUCH TO BE DESIRED. does not NEG_heat N...
dtype: object

#### Comparing length of review that is preprocessed with length of review before preprocessing

In [25]:
len_after_preprocessing = reviews_processed.apply(len).mean()

print(f"{len_after_cleaning}, {len_after_preprocessing}")

337.23428333333334, 249.59886666666668


In [26]:
print("Sample of preprocessed Data:")
reviews_processed.head(10)

Sample of preprocessed Data:


0    PICTURE SHOW 17 TITEL SAY 13 PC RECEIVE 12 . u...
1           KID LIKE NAME . horrid name kid like name 
2    TWO STAR . thick could get cheap local beauty ...
3    FAKE . not NEG_order NEG_this NEG_product NEG_...
4    NOT BUY . waxed not NEG_new NEG_to NEG_painful...
5    NOT LOOK LIKE PICTURE . scarf not look anythin...
6    BAD EXPERIENCE . friend buy sunscreen know yea...
7    UNDERWHELMED . develop fondness june jacob gre...
8    BURN GUM . stuff work careful actually burn gu...
9    LEAF MUCH DESIRE . not NEG_heat NEG_up NEG_eno...
dtype: object

# TF-IDF Feature Extraction

In [27]:
from sklearn.model_selection import train_test_split
X_train_vanilla, X_test_vanilla, Y_train_vanilla, Y_test_vanilla = train_test_split(reviews_vanilla, df['class'], test_size = 0.2, random_state = 0, stratify=df['class'])

In [28]:
X_train_processed, X_test_processed, Y_train_processed, Y_test_processed = train_test_split(reviews_pr  ocessed, df['class'], test_size = 0.2, random_state = 0, stratify=df['class'])

In [29]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf_idf_vect_vanilla = TfidfVectorizer(use_idf=True, max_features=5000)
X_train_tfidf_vanilla = tf_idf_vect_vanilla.fit_transform(X_train_vanilla)
X_test_tfidf_vanilla = tf_idf_vect_vanilla.transform(X_test_vanilla)

In [30]:
print("For cleaned/vanilla data:")
print(f"\tX_train shape: {X_train_tfidf_vanilla.shape}")
print(f"\tY_train shape: {Y_train_vanilla.shape}\n")

print(f"\tX_test shape: {X_test_tfidf_vanilla.shape}")
print(f"\tY_test shape: {Y_test_vanilla.shape}\n")

For cleaned/vanilla data:
	X_train shape: (48000, 5000)
	Y_train shape: (48000,)

	X_test shape: (12000, 5000)
	Y_test shape: (12000,)



In [31]:
tf_idf_vect_processed = TfidfVectorizer(use_idf=True, max_features=5000)
X_train_tfidf_processed = tf_idf_vect_processed.fit_transform(X_train_processed)
X_test_tfidf_processed = tf_idf_vect_processed.transform(X_test_processed)

In [32]:
print("For pre-processed data:")
print(f"\tX_train shape: {X_train_tfidf_processed.shape}")
print(f"\tY_train shape: {Y_train_processed.shape}\n")

print(f"\tX_test shape: {X_test_tfidf_processed.shape}")
print(f"\tY_test shape: {Y_test_processed.shape}\n")

For pre-processed data:
	X_train shape: (48000, 5000)
	Y_train shape: (48000,)

	X_test shape: (12000, 5000)
	Y_test shape: (12000,)



# Perceptron

### Without Stop Word Removal and Without Lemmatization

In [33]:
from sklearn.linear_model import Perceptron

prctrn_vanilla = Perceptron(tol=1e-3, random_state=0, validation_fraction=0.3)
prctrn_vanilla.fit(X_train_tfidf_vanilla, Y_train_vanilla)

# training accuracy
print(prctrn_vanilla.score(X_train_tfidf_vanilla, Y_train_vanilla))

# testing accuracy
print(prctrn_vanilla.score(X_test_tfidf_vanilla, Y_test_vanilla))

0.7999166666666667
0.7331666666666666


In [34]:
# classification report
prctrn_pred_vanilla = prctrn_vanilla.predict(X_test_tfidf_vanilla)
print(classification_report(Y_test_vanilla, prctrn_pred_vanilla))

# class-wise precision, recall and f1-score
prctrn_report_vanilla = classification_report(Y_test_vanilla, prctrn_pred_vanilla, output_dict=True)
print(f"{prctrn_report_vanilla['1']['precision']}, {prctrn_report_vanilla['1']['recall']}, {prctrn_report_vanilla['1']['f1-score']}")
print(f"{prctrn_report_vanilla['2']['precision']}, {prctrn_report_vanilla['2']['recall']}, {prctrn_report_vanilla['2']['f1-score']}")
print(f"{prctrn_report_vanilla['3']['precision']}, {prctrn_report_vanilla['3']['recall']}, {prctrn_report_vanilla['3']['f1-score']}")
print(f"{prctrn_report_vanilla['macro avg']['precision']}, {prctrn_report_vanilla['macro avg']['recall']}, {prctrn_report_vanilla['macro avg']['f1-score']}")

              precision    recall  f1-score   support

           1       0.74      0.73      0.73      4000
           2       0.67      0.63      0.65      4000
           3       0.78      0.84      0.81      4000

    accuracy                           0.73     12000
   macro avg       0.73      0.73      0.73     12000
weighted avg       0.73      0.73      0.73     12000

0.7402696514881709, 0.7275, 0.7338292775185978
0.6719083155650319, 0.63025, 0.6504127966976265
0.7799397729905027, 0.84175, 0.8096669472165444
0.7307059133479018, 0.7331666666666666, 0.7313030071442562


### With Stop Word Removal and Lemmatization

In [35]:
prctrn_processed = Perceptron(tol=1e-3, random_state=0, validation_fraction=0.3)
prctrn_processed.fit(X_train_tfidf_processed, Y_train_processed)

# training accuracy
print(prctrn_processed.score(X_train_tfidf_processed, Y_train_processed))

# testing accuracy
print(prctrn_processed.score(X_test_tfidf_processed, Y_test_processed))

0.7800416666666666
0.7121666666666666


In [36]:
# classification report
prctrn_pred_processed = prctrn_processed.predict(X_test_tfidf_processed)
print(classification_report(Y_test_processed, prctrn_pred_processed))

# class-wise precision, recall and f1-score
prctrn_report_processed = classification_report(Y_test_processed, prctrn_pred_processed, output_dict=True)
print(f"{prctrn_report_processed['1']['precision']}, {prctrn_report_processed['1']['recall']}, {prctrn_report_processed['1']['f1-score']}")
print(f"{prctrn_report_processed['2']['precision']}, {prctrn_report_processed['2']['recall']}, {prctrn_report_processed['2']['f1-score']}")
print(f"{prctrn_report_processed['3']['precision']}, {prctrn_report_processed['3']['recall']}, {prctrn_report_processed['3']['f1-score']}")
print(f"{prctrn_report_processed['macro avg']['precision']}, {prctrn_report_processed['macro avg']['recall']}, {prctrn_report_processed['macro avg']['f1-score']}")

              precision    recall  f1-score   support

           1       0.68      0.77      0.72      4000
           2       0.66      0.57      0.61      4000
           3       0.79      0.79      0.79      4000

    accuracy                           0.71     12000
   macro avg       0.71      0.71      0.71     12000
weighted avg       0.71      0.71      0.71     12000

0.6790177592633194, 0.77425, 0.7235136082233383
0.6633663366336634, 0.5695, 0.6128598331988162
0.7917602996254681, 0.79275, 0.7922548407245471
0.7113814651741502, 0.7121666666666666, 0.7095427607155672


# SVM

### Without Stop Word Removal and Without Lemmatization

In [37]:
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

SVM_vanilla = LinearSVC()
SVM_vanilla.fit(X_train_tfidf_vanilla, Y_train_vanilla)

# training accuracy
print(SVM_vanilla.score(X_train_tfidf_vanilla, Y_train_vanilla))

# test accuracy
print(SVM_vanilla.score(X_test_tfidf_vanilla, Y_test_vanilla))

0.843125
0.77325


In [38]:
# classification report
SVM_pred_vanilla = SVM_vanilla.predict(X_test_tfidf_vanilla)
print(classification_report(Y_test_vanilla, SVM_pred_vanilla))

# class-wise precision, recall and f1-score
SVM_report_vanilla = classification_report(Y_test_vanilla, SVM_pred_vanilla, output_dict=True)
print(f"{SVM_report_vanilla['1']['precision']}, {SVM_report_vanilla['1']['recall']}, {SVM_report_vanilla['1']['f1-score']}")
print(f"{SVM_report_vanilla['2']['precision']}, {SVM_report_vanilla['2']['recall']}, {SVM_report_vanilla['2']['f1-score']}")
print(f"{SVM_report_vanilla['3']['precision']}, {SVM_report_vanilla['3']['recall']}, {SVM_report_vanilla['3']['f1-score']}")
print(f"{SVM_report_vanilla['macro avg']['precision']}, {SVM_report_vanilla['macro avg']['recall']}, {SVM_report_vanilla['macro avg']['f1-score']}")

              precision    recall  f1-score   support

           1       0.77      0.78      0.78      4000
           2       0.70      0.68      0.69      4000
           3       0.84      0.86      0.85      4000

    accuracy                           0.77     12000
   macro avg       0.77      0.77      0.77     12000
weighted avg       0.77      0.77      0.77     12000

0.7717256746719485, 0.77925, 0.7754695857693744
0.7048673705897502, 0.68425, 0.6944056831155653
0.8398724865129966, 0.85625, 0.8479821738053974
0.7721551772582318, 0.77325, 0.7726191475634456


### With Stop Word Removal and Lemmatization

In [39]:
SVM_processed = LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True, intercept_scaling=1, loss='squared_hinge', max_iter=1000, multi_class='ovr', penalty='l2', random_state=None, tol=0.0001, verbose=0)
SVM_processed.fit(X_train_tfidf_processed, Y_train_processed)

# training accuracy
print(SVM_processed.score(X_train_tfidf_processed, Y_train_processed))

# test accuracy
print(SVM_processed.score(X_test_tfidf_processed, Y_test_processed))

0.8331458333333334
0.7536666666666667


In [40]:
# classification report
SVM_pred_processed = SVM_processed.predict(X_test_tfidf_processed)
print(classification_report(Y_test_processed, SVM_pred_processed))

# class-wise precision, recall and f1-score
SVM_report_processed = classification_report(Y_test_processed, SVM_pred_processed, output_dict=True)
print(f"{SVM_report_processed['1']['precision']}, {SVM_report_processed['1']['recall']}, {SVM_report_processed['1']['f1-score']}")
print(f"{SVM_report_processed['2']['precision']}, {SVM_report_processed['2']['recall']}, {SVM_report_processed['2']['f1-score']}")
print(f"{SVM_report_processed['3']['precision']}, {SVM_report_processed['3']['recall']}, {SVM_report_processed['3']['f1-score']}")
print(f"{SVM_report_processed['macro avg']['precision']}, {SVM_report_processed['macro avg']['recall']}, {SVM_report_processed['macro avg']['f1-score']}")

              precision    recall  f1-score   support

           1       0.75      0.76      0.76      4000
           2       0.68      0.65      0.67      4000
           3       0.82      0.84      0.83      4000

    accuracy                           0.75     12000
   macro avg       0.75      0.75      0.75     12000
weighted avg       0.75      0.75      0.75     12000

0.750122970978849, 0.7625, 0.7562608480039673
0.6826597131681877, 0.6545, 0.6682833439693681
0.8236155159795072, 0.844, 0.8336831707618224
0.7521327333755147, 0.7536666666666666, 0.7527424542450527


# Logistic Regression

### Without Stop Word Removal and Without Lemmatization

In [41]:
from sklearn.linear_model import LogisticRegression

lr_vanilla = LogisticRegression(multi_class='multinomial', max_iter=10000)
lr_vanilla.fit(X_train_tfidf_vanilla, Y_train_vanilla)

# training accuracy
print(lr_vanilla.score(X_train_tfidf_vanilla, Y_train_vanilla))

# test accuracy
print(lr_vanilla.score(X_test_tfidf_vanilla, Y_test_vanilla))

0.8291041666666666
0.78325


In [42]:
# classification report
lr_pred_vanilla = lr_vanilla.predict(X_test_tfidf_vanilla)
print(classification_report(Y_test_vanilla, lr_pred_vanilla))

# class-wise precision, recall and f1-score
lr_report_vanilla = classification_report(Y_test_vanilla, lr_pred_vanilla, output_dict=True)
print(f"{lr_report_vanilla['1']['precision']}, {lr_report_vanilla['1']['recall']}, {lr_report_vanilla['1']['f1-score']}")
print(f"{lr_report_vanilla['2']['precision']}, {lr_report_vanilla['2']['recall']}, {lr_report_vanilla['2']['f1-score']}")
print(f"{lr_report_vanilla['3']['precision']}, {lr_report_vanilla['3']['recall']}, {lr_report_vanilla['3']['f1-score']}")
print(f"{lr_report_vanilla['macro avg']['precision']}, {lr_report_vanilla['macro avg']['recall']}, {lr_report_vanilla['macro avg']['f1-score']}")

              precision    recall  f1-score   support

           1       0.78      0.78      0.78      4000
           2       0.71      0.71      0.71      4000
           3       0.86      0.85      0.86      4000

    accuracy                           0.78     12000
   macro avg       0.78      0.78      0.78     12000
weighted avg       0.78      0.78      0.78     12000

0.7806645016237822, 0.78125, 0.7809571410720979
0.7085813492063492, 0.71425, 0.7114043824701196
0.8617906683480454, 0.85425, 0.8580037664783428
0.7836788397260589, 0.7832500000000001, 0.78345509667352


### With Stop Word Removal and Lemmatization

In [43]:
lr_processed = LogisticRegression(multi_class='multinomial', max_iter=10000)
lr_processed.fit(X_train_tfidf_processed, Y_train_processed)

# training accuracy
print(lr_processed.score(X_train_tfidf_processed, Y_train_processed))

# test accuracy
print(lr_processed.score(X_test_tfidf_processed, Y_test_processed))

0.8192916666666666
0.7645


In [44]:
# classification report
lr_pred_processed = lr_processed.predict(X_test_tfidf_processed)
print(classification_report(Y_test_processed, lr_pred_processed))

# class-wise precision, recall and f1-score
lr_pred_processed = lr_processed.predict(X_test_tfidf_processed)

lr_report_processed = classification_report(Y_test_processed, lr_pred_processed, output_dict=True)
print(f"{lr_report_processed['1']['precision']}, {lr_report_processed['1']['recall']}, {lr_report_processed['1']['f1-score']}")
print(f"{lr_report_processed['2']['precision']}, {lr_report_processed['2']['recall']}, {lr_report_processed['2']['f1-score']}")
print(f"{lr_report_processed['3']['precision']}, {lr_report_processed['3']['recall']}, {lr_report_processed['3']['f1-score']}")
print(f"{lr_report_processed['macro avg']['precision']}, {lr_report_processed['macro avg']['recall']}, {lr_report_processed['macro avg']['f1-score']}")

              precision    recall  f1-score   support

           1       0.77      0.77      0.77      4000
           2       0.69      0.69      0.69      4000
           3       0.84      0.84      0.84      4000

    accuracy                           0.76     12000
   macro avg       0.76      0.76      0.76     12000
weighted avg       0.76      0.76      0.76     12000

0.766541822721598, 0.7675, 0.7670206121174266
0.6879076768690416, 0.6855, 0.6867017280240422
0.8386131204789224, 0.8405, 0.8395555000624297
0.764354206689854, 0.7645, 0.7644259467346327


# Naive Bayes

### Without Stop Word Removal and Without Lemmatization

In [45]:
from sklearn.naive_bayes import MultinomialNB

MNB_vanilla = MultinomialNB()
MNB_vanilla.fit(X_train_tfidf_vanilla, Y_train_vanilla)

# training accuracy
print(MNB_vanilla.score(X_train_tfidf_vanilla, Y_train_vanilla))

# test accuracy
print(MNB_vanilla.score(X_test_tfidf_vanilla, Y_test_vanilla))

0.7667083333333333
0.7415833333333334


In [46]:
# classification report
MNB_pred_vanilla = MNB_vanilla.predict(X_test_tfidf_vanilla)
print(classification_report(Y_test_vanilla, MNB_pred_vanilla))

# class-wise precision, recall and f1-score
MNB_report_vanilla = classification_report(Y_test_vanilla, MNB_pred_vanilla, output_dict=True)
print(f"{MNB_report_vanilla['1']['precision']}, {MNB_report_vanilla['1']['recall']}, {MNB_report_vanilla['1']['f1-score']}")
print(f"{MNB_report_vanilla['2']['precision']}, {MNB_report_vanilla['2']['recall']}, {MNB_report_vanilla['2']['f1-score']}")
print(f"{MNB_report_vanilla['3']['precision']}, {MNB_report_vanilla['3']['recall']}, {MNB_report_vanilla['3']['f1-score']}")
print(f"{MNB_report_vanilla['macro avg']['precision']}, {MNB_report_vanilla['macro avg']['recall']}, {MNB_report_vanilla['macro avg']['f1-score']}")

              precision    recall  f1-score   support

           1       0.76      0.74      0.75      4000
           2       0.66      0.68      0.67      4000
           3       0.80      0.81      0.80      4000

    accuracy                           0.74     12000
   macro avg       0.74      0.74      0.74     12000
weighted avg       0.74      0.74      0.74     12000

0.7639462809917356, 0.7395, 0.7515243902439025
0.6599465630313335, 0.67925, 0.6694591597880991
0.8037895786586886, 0.806, 0.804893271751342
0.742560807560586, 0.7415833333333334, 0.7419589405944479


### With Stop Word Removal and Lemmatization

In [47]:
from sklearn.naive_bayes import MultinomialNB

MNB_processed = MultinomialNB()
MNB_processed.fit(X_train_tfidf_processed, Y_train_processed)

# train accuracy
print(MNB_processed.score(X_train_tfidf_processed, Y_train_processed))

# test accuracy
print(MNB_processed.score(X_test_tfidf_processed, Y_test_processed))

0.7568958333333333
0.7286666666666667


In [48]:
# classification report
MNB_pred_processed = MNB_processed.predict(X_test_tfidf_processed)
print(classification_report(Y_test_processed, MNB_pred_processed))

# class-wise precision, recall and f1-score
MNB_report_processed = classification_report(Y_test_processed, MNB_pred_processed, output_dict=True)
print(f"{MNB_report_processed['1']['precision']}, {MNB_report_processed['1']['recall']}, {MNB_report_processed['1']['f1-score']}")
print(f"{MNB_report_processed['2']['precision']}, {MNB_report_processed['2']['recall']}, {MNB_report_processed['2']['f1-score']}")
print(f"{MNB_report_processed['3']['precision']}, {MNB_report_processed['3']['recall']}, {MNB_report_processed['3']['f1-score']}")
print(f"{MNB_report_processed['macro avg']['precision']}, {MNB_report_processed['macro avg']['recall']}, {MNB_report_processed['macro avg']['f1-score']}")

              precision    recall  f1-score   support

           1       0.76      0.73      0.74      4000
           2       0.65      0.66      0.65      4000
           3       0.78      0.80      0.79      4000

    accuracy                           0.73     12000
   macro avg       0.73      0.73      0.73     12000
weighted avg       0.73      0.73      0.73     12000

0.756078634247284, 0.73075, 0.7431985761505213
0.6491228070175439, 0.65675, 0.6529141294892506
0.781502324443357, 0.7985, 0.7899097316681093
0.7289012552360616, 0.7286666666666667, 0.7286741457692938
